In [1]:
import numpy as np 
from queue import Queue

import copy

In [2]:
def algorithm_s1(current_sequence):
    
    poss_next_states = []
    
    current_sequence_cp = copy.deepcopy(current_sequence)
    
    for i in range(len(current_sequence_cp)):
        
        for j in range(i+1, len(current_sequence_cp)):
            # make sure the two states have no same letters (so that an input isnt repeated in the combined state)
            if len(set(current_sequence_cp[i]) & set(current_sequence_cp[j])) == 0: 
                
                next_possible = ''.join(sorted(current_sequence_cp[i] + current_sequence_cp[j]))
        
                if next_possible not in current_sequence:
                    poss_next_states.append(next_possible) # if the item is a new one, add it into next_item_set
                    current_sequence_cp = current_sequence_cp[:j] # delete everything after j including j, as the combination of those item with item after current i would be larger than ij
                    break
                    
    
    poss_next_states = set(poss_next_states)
   
    poss_next_states = algorithm_s2(poss_next_states,current_sequence) # filter the set to remove states that violate the contraints imposed by order of the previous states

    return poss_next_states



def algorithm_s1_NT(current_sequence):
    '''
    equivalent to algrithm S1 but reframed the filtering so the next states are entirely generated from the sequence. I think this is equivalent but more convincing that we 
    havent just hardcoded the constraints in
    '''
    
    poss_next_states = {} # dictionary which hold the possible next states and the indexs at which they were generated 
    
 
    for i in range(len(current_sequence)):
        
        for j in range(i+1, len(current_sequence)):
            # make sure the two states have no same letters (so that an input isnt repeated in the combined state)
            if len(set(current_sequence[i]) & set(current_sequence[j])) == 0: 
                
                next_possible = ''.join(sorted(current_sequence[i] + current_sequence[j]))
        
                if next_possible not in current_sequence: # if the item is a new one, add it into next_item_set
                    poss_next_states[next_possible] = poss_next_states.get(next_possible, [])
                    poss_next_states[next_possible].append([i,j])
                    
                    
                    
 
    poss_next_states = algorithm_s2_NT(poss_next_states) # filter the set to remove states that violate the contraints imposed by order of the previous states
    
    return poss_next_states


            
            
            
    

In [3]:
def algorithm_s2(possible_next_states, current_sequence):
    
    possible_next_states_l = copy.deepcopy(list(possible_next_states))

    for i in range(len(possible_next_states_l)):
        
        for j in range(i+1, len(possible_next_states_l)):
            p1 = possible_next_states_l[i]
            p2 = possible_next_states_l[j]
            
            # check to see if there are any common inputs and that these states havent previously been removed
            if len(set(p1) & set(p2)) > 0 and p1 in possible_next_states and p2 in possible_next_states:
                 
                # the inputs that are different between these two states
                r1 = set(p1) - set(p2)
                r2 = set(p2) - set(p1)
                
                # if either one of these are empty we know the other must have a higher conc so cant go next 
                if len(r1) == 0:
                    possible_next_states.remove(p2)
                
                elif len(r2) == 0:
                    possible_next_states.remove(p1)
                
                    
                else: #otherwise we need to check against the current sequence 
                    
                    str1 = ''.join(sorted(r1))
                    str2 = ''.join(sorted(r2))
                 
                    if current_sequence.index(str1) < current_sequence.index(str2):
                        possible_next_states.remove(p2)
                    else:
                        possible_next_states.remove(p1)
                        
                        
    return possible_next_states


def algorithm_s2_NT(possible_next_states):
    
    '''
    this looks at all given combinations of indices that generate each possible state and eliminates those which cannot go next.
    
    e.g. for [b, d, a, bd, c, ab, ad]
    two possible next states are bc, which can be generated from indices [0, 4] and abd which can be generated from indices [2, 3] and [1, 5]. 
    Looking at the indices [0, 4] and [1, 5] we know that abd must be higher conc than bc becuase 1 > 0 and 5 > 4 i.e. d > b and ad > c therefore abd > bc
    This means that abd cannot be a valid next state as it is constrained to be higher than bc
    '''
    
    states = list(possible_next_states.keys())
    
    valid_states = []
    
    '''
    brute force through all indices to find not possible orders. There is probably a much better way to do this, but isnt too slow for 5 inputs 
    '''
    for i in range(len(states)):
        s1 = states[i]
        
        possible = True
        
        i1s = possible_next_states[s1]
        
        for j in range(0, len(states)):
            
            if i == j:
                continue
            s2 = states[j]
            
            i2s = possible_next_states[s2]
            
            for i1 in i1s:
                for i2 in i2s:
                    
                    if i2[0]<=i1[0] and i2[1]<=i1[1]:
                        possible = False
                        break
                        
                if not possible:
                    break
                    
            if not possible:
                break
                
    
            
        if possible:
            valid_states.append(s1)
            
            
    return set(valid_states)
    

In [4]:
def algorithm_s3(n_inputs):
    
    single_inputs = set(list(map(chr, range(97, 97+n_inputs))))
    
    queue = Queue(maxsize=0)
    queue.put([])
    
    while True:
        sequence = queue.get()
        
        # next possible states are the remaining single inputs and those returns by algorithm s1
        possible_next = (single_inputs - set(sequence)) | algorithm_s1(sequence)
        
        if len(possible_next) == 0:
            queue.put(possible_next)
            return list(queue.queue)

        else:
            possible_next_l = list(possible_next)
            
            for next_item in possible_next_l:
                queue.put(sequence + [next_item])
                
                
def algorithm_s3_NT(n_inputs):
    
    single_inputs = set(list(map(chr, range(97, 97+n_inputs))))
    
    queue = Queue(maxsize=0)
    queue.put([])
    
    while True:
        sequence = queue.get()
        
        # next possible states are the remaining single inputs and those returns by algorithm s1
        possible_next = (single_inputs - set(sequence)) | algorithm_s1_NT(sequence)
        
        if len(possible_next) == 0:
            queue.put(possible_next)
            return list(queue.queue)

        else:
            possible_next_l = list(possible_next)
            
            for next_item in possible_next_l:
                queue.put(sequence + [next_item])

    
    
    

# below agrees with table 2 of the supplementary 

In [5]:
for n_inputs in range(6):

    allowable_orders = algorithm_s3_NT(n_inputs)
    print('n_inputs: {}, allowable orders: {}'.format(n_inputs, len(allowable_orders)))

n_inputs: 0, allowable orders: 1
n_inputs: 1, allowable orders: 1
n_inputs: 2, allowable orders: 2
n_inputs: 3, allowable orders: 12
n_inputs: 4, allowable orders: 336
n_inputs: 5, allowable orders: 65520


In [6]:
for n_inputs in range(6):

    allowable_orders = algorithm_s3(n_inputs)
    print('n_inputs: {}, allowable orders: {}'.format(n_inputs, len(allowable_orders)))

n_inputs: 0, allowable orders: 1
n_inputs: 1, allowable orders: 1
n_inputs: 2, allowable orders: 2
n_inputs: 3, allowable orders: 12
n_inputs: 4, allowable orders: 336
n_inputs: 5, allowable orders: 65520
